In [88]:
# Load libraries
import gurobipy as gp
from gurobipy import GRB,tuplelist

In [89]:
# Set parameters
P = 0
p = -5   # Defining the accuracy
set_l = range(p,P+1)
set_k = range(10)
set_kl = tuplelist([(k,l) for l in set_l for k in set_k])

m = gp.Model('MDT')

# Define the continuous variables
x1 = m.addVar(lb=0, ub=1.5, vtype=GRB.CONTINUOUS, name="x1")
x2 = m.addVar(lb=0, ub=1.5, vtype=GRB.CONTINUOUS, name="x2")

# print([i for i in set_kl])

In [90]:
m.update()

w12 = m.addVar(vtype=GRB.CONTINUOUS, name="w12")
Delta_w12 = m.addVar(lb=0, ub=1.5 * x1.UB, vtype=GRB.CONTINUOUS, name="Delta_w12")  # 1.5 * 1.5 upper bound
Delta_x1 = m.addVar(lb=0, ub=1.5 * 10**p, vtype=GRB.CONTINUOUS, name="Delta_x1")

In [91]:
# Define the indexed continuous variables (hat_x_k) and binary variables (z_k)
hat_x = m.addVars(set_kl, lb=0, ub=1.5, vtype=GRB.CONTINUOUS, name="hat_x")
z = m.addVars(set_kl, vtype=GRB.BINARY, name="z")

# Set the objective function: Minimize f = -x1 + w12 - x2
m.setObjective(-x1 + w12 - x2, GRB.MINIMIZE)
m.update()
# hat_x

In [92]:
# Constraint 1: -6*x1 + 8*x2 <= 3
m.addConstr(-6*x1 + 8*x2 <= 3, "c1")

# Constraint 2: 3*x1 - x2 <= 3
m.addConstr(3*x1 - x2 <= 3, "c2")

# Constraint 3: w12 = sum(k * hat_x[k]) + Delta_w12
m.addConstr(w12 == gp.quicksum(gp.quicksum(10**l * k * hat_x[k,l] for k in set_k) for l in set_l) + Delta_w12, "c3")

# Constraint 4: x1 = sum(10^0 * k * z_k) + Delta_x1
m.addConstr(x1 == gp.quicksum(gp.quicksum(10**l * k * z[k,l] for k in set_k) for l in set_l) + Delta_x1, "c4")

# Constraint 5: x2 = sum(hat_x[k])
m.addConstrs((x2 == gp.quicksum(hat_x[k,l] for k in set_k) for l in set_l), "c5")

# Constraint 6: hat_x[k] <= 1.5 * z[k] for all k
# for k in range(10):
m.addConstrs((hat_x[k,l] <= 1.5 * z[k,l] for k,l in set_kl), "c6")

# Constraint 7: Sum(z_k) = 1
m.addConstrs((gp.quicksum(z[k,l] for k in set_k) == 1 for l in set_l), "c7")

# Constraint 8: 0 <= Delta_w12 <= 1.5 * Delta_x1
m.addConstr(Delta_w12 >= 0, "c8a")
m.addConstr(Delta_w12 <= 1.5 * Delta_x1, "c8b")

# Constraint 9: (x2 - 1.5) * 10^0 + 1.5 * Delta_x1 <= Delta_w12 <= x2 * 10^0
m.addConstr((x2 - 1.5) * 10**p + 1.5 * Delta_x1 <= Delta_w12, "c9a")
m.addConstr(Delta_w12 <= x2 * 10**p, "c9b")

# Constraint 10: 0 <= Delta_x1 <= 1.5 * 10^0
m.addConstr(Delta_x1 >= 0, "c10a")
m.addConstr(Delta_x1 <= 1.5 * 10**p, "c10b")

# Constraint 11: 0 <= hat_x[k] <= 1.5 for all k
# for k in range(10):
#     m.addConstr(hat_x[k] >= 0, f"c11a_{k}")
#     m.addConstr(hat_x[k] <= 1.5, f"c11b_{k}")

# Optimize the m
m.Params.OutputFlag = 1
m.write('kk.lp')
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26120.2))

CPU model: AMD Ryzen 7 2700X Eight-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 82 rows, 125 columns and 372 nonzeros
Model fingerprint: 0xc95bf073
Variable types: 65 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e-05, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-05, 2e+00]
  RHS range        [2e-05, 3e+00]
Found heuristic solution: objective -0.9112900
Presolve removed 19 rows and 23 columns
Presolve time: 0.00s
Presolved: 63 rows, 102 columns, 263 nonzeros
Variable types: 56 continuous, 46 integer (46 binary)

Root relaxation: objective -1.775501e+00, 51 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

 

     0     0   -1.11373    0    7   -1.08000   -1.11373  3.12%     -    0s
H    0     0                      -1.0801995   -1.11373  3.10%     -    0s
H    0     0                      -1.0815887   -1.11373  2.97%     -    0s
     0     2   -1.11373    0    6   -1.08159   -1.11373  2.97%     -    0s
H    9     5                      -1.0817027   -1.11373  2.96%  10.1    0s
H   19     7                      -1.0817041   -1.11078  2.69%   6.5    0s
H   20     7                      -1.0828037   -1.11078  2.58%   6.5    0s
H   32    11                      -1.0832031   -1.10000  1.55%   4.7    0s
H   35    11                      -1.0833034   -1.09545  1.12%   4.3    0s
H   43    12                      -1.0833204   -1.09074  0.68%   3.8    0s
H   55    17                      -1.0833314   -1.08619  0.26%   3.2    0s
*   81    31              15      -1.0833363   -1.08619  0.26%   2.8    0s
H  100    37                      -1.0833364   -1.08591  0.24%   2.6    0s
H  101    37             

In [99]:
print('There are', m.NumVars, 'variables and', m.NumConstrs, 'constraints')
print('The program runs for', m.Runtime, 'seconds')

There are 125 variables and 82 constraints
The program runs for 0.18700003623962402 seconds


In [94]:
Delta_x1

<gurobi.Var Delta_x1 (value 6.666666666666668e-06)>